# Projeto 1 - Ciência dos Dados

Nome: João Otávio Palma

Nome: Pedro Garcia Carneiro da Cunha

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
## Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Classificador automático sobre a Bolsa de valores brasileira (B3)


As notícias tiveram o seu escopo dentro da parte de economia do IstoéDinheiro. Com isso, o foco o target foi a bolsa brasileira, avaliando as informações nacionais e internacionais que poderiam causar um aumento da bolsa ou uma queda da mesma. Dessa forma, houve uma separação entre assuntos irrelevantes(pouco impactantes ou com nenhum realcionamento com o target), neutros(indiferença na manipulação da bolsa de valores brasileira) e relevantes(possuem chance de gerar alguma alteração). 

## "Bloco" de funções que auxiliaram o classificador automático:

### Cleanup --> retira caracteres especiais e os caracteres que encontram-se em Caps lock:

In [2]:
def cleanup(text):

    #importando a string
    punctuation = '[\n!-:.?;,")(~''/|\*&%@#$_><}{ºª+=`]' 
    
    pattern = re.compile(punctuation)
    
    text_subbed = re.sub(pattern, '', text)
    
    return text_subbed.lower()
## Teste
###textinho = 'hoje eu e o pedrão fomos ~ao shopping? talvez sim, talvez nao! \n mas quem sabe - algumas pessoas foram: magali, cebolinha, magazine luiza (guilda)'

###txtinho = cleanup(textinho)
###print(txtinho)

### NLP (Natural Language Processing) --> uso de bibliotecas que tenham stopwords para melhorar o processamento do calssificador:

In [3]:
def stpwd (l_p): # recebe uma lista de palavras para analisar
    
    l_p_filtradas = []
    
    p_stpwd = stopwords.words('portuguese') # Acessa a lista de "stop words" da língua portuguesa dentro da biblioteca NLTK
    
    for p in l_p: # p = palarvra
        
        if p not in p_stpwd: # verifica se a palavra não está presente na lista
            
            l_p_filtradas.append(p)
    
    return l_p_filtradas

### Função de Laplace --> Realiza  a suavização dos valores para evitar falhas do contador.

In [4]:
def lp (word, cat, port):
    
    α = 1
    
    if word in cat.index:
        
        freq_abs = cat.loc[word]
        
    else:
        
        freq_abs = 0
        
    tot = len(cat)
    
    rept = freq_abs
    
    outer_elements = len(port)
    
    # Aqui entra a fórmula de suavização de Laplace
    
    lp_suav = (rept + α) / (tot + outer_elements + α)
    
    return lp_suav

## Verificando onde o arquivo se encontra:

In [5]:
print('Esperamos trabalhar no diretório:')
print(os.getcwd())

Esperamos trabalhar no diretório:
C:\Users\pedro\Documents\GitHub\22-2b-cd-p1-grupo_joaoogp\notebooks


## Carregando a base de dados com as notícias classificadas manualmente:

In [6]:
os.listdir('data')

['dados.xlsx', '~$dados.xlsx']

In [7]:
filename = 'data/dados.xlsx'

# Verificando se é possível achar o arquivo:

if 'dados.xlsx' in os.listdir('data'):
    print(f'Parece que o arquivo {filename} está na pasta diretório correta')
    
else:
    print(f'Parece que o arquivo {filename} NÃO está na pasta diretório correta')
    

Parece que o arquivo data/dados.xlsx está na pasta diretório correta


In [8]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target
0,economia,"Investimento Direto no País soma US$ 4,483 bi ...",Os Investimentos Diretos no País (IDP) somaram...,26/08/2022 10:23,47,2
1,economia,‘Não queremos a chinesada entrando aqui quebra...,"O ministro da Economia, Paulo Guedes, afirmou ...",26/08/2022 17:08,46,0
2,economia,Presidente de petroleira russa morre ao cair d...,"MOSCOU (Reuters) – Ravil Maganov, presidente d...",01/09/2022 16:51,22,1
3,economia,"PPI da Alemanha avança 37,2% em julho na compa...","O índice de preços ao produtor (PPI, na sigla ...",19/08/2022 07:19,72,1
4,economia,Problema em traje de cosmonauta russo força fi...,Por Joey Roulette WASHINGTON (Reuters) – Um do...,17/08/2022 16:47,78,1


In [36]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target
0,economia,Bolsonaro cita 1964 e ano do impeachment de Di...,Antes de sair do Palácio da Alvorada nesta qua...,07/09/2022 09:40,8,1
1,economia,Investidores não veem guinada do Fed e se prep...,Por Gertrude Chavez-Dreyfuss NOVA YORK (Reuter...,24/08/2022 08:10,57,1
2,economia,Ministro diz que enviou estudo do Porto de San...,"O ministro da Infraestrutura, Marcelo Sampaio,...",01/09/2022 13:06,24,0
3,economia,Missão da AIEA na usina de Zaporizhzhia “pode ...,KIEV (Reuters) – A missão de monitoramento da ...,01/09/2022 08:30,26,1
4,economia,Bancos admitem ‘licença’ de R$ 70 bilhões,Representantes de bancos e fundos de investime...,16/08/2022 17:00,82,2


### Classificador automático

No projeto 1, o ojetivo central do trabalho é criar um classificador de notícias baseado na descrição das mesmas. O material utilizado foi as notícias do site IstoÉ Dinheiro, onde as notícias envolvem tópicos relacionados à economia mundial. Com isso, a classificação foi feita do seguinte modo:

0 - Notícias NEGATIVAS, ou seja, aquelas que interferem negativamente na bolsa de valores brasileira.
1 - Notícias NEUTRAS, ou seja, aquelas que não impactam diretamente na bolsa de valores do Brasil.
2 - Notícias POSITIVAS, ou seja, aquelas que interferem positivamente na bolsa de valores brasileira.

Para realizar esse processo, o Teorema de Bayes foi utilizado para desenvolver um classificador Naive-Bayes. Ele consiste em examinar a ocorrência das palavras em uma dada categoria e determina a probabilidade de uma dada nova informação não classificada ser de uma categoria determinada a partir das palavras-chave da notícia.

A base de dados de treinamento do algoritmo que foi utilizada foi uma série de notícicas classificadas manualmente. Isso serviu para o classificador determinar a frequência das palavras como dito anteriormente.

O desenvolvimento do classificador consistiu em alguns cálculos de parâmetros que seriam utilizados no Teorema de Bayes:

1.  $P(Notícia|Positiva)$: probabilidade de encontrar a notícia dentre as notícias Positivas;
2.  $P(Notícia|Neutra)$: probabilidade de encontrar a notícia dentre as notícias Neutras;
3.  $P(Notícia|Negativa)$: probabilidade de encontrar a notícia dentre as notícias Negativas;
4.  $P(Positiva)$: probabilidade de ser Positiva;
5.  $P(Neutra)$: probabilidade de ser Neutra;
6.  $P(Negativa)$: probabilidade de ser Negativa;

Com as variáveis já obtidas, tornou-se possível o cálculo das probabilidades de classificação das informações:

* *Probabilidade de ser Positiva dado a notícia*:
$$P(Positiva|Notícia) = \frac{P(Notícia|Positiva) P(Positiva)}{P(Notícia)}$$

* *Probabilidade de ser Negativa dado a notícia*:
$$P(Negativa|Notícia) = \frac{P(Notícia|Negativa) P(Negativa)}{P(Notícia)}$$

* *Probabilidade de ser Neutra dado a notícia*:
$$P(Neutra|Notícia) = \frac{P(Notícia|Neutra) P(Neutra)}{P(Notícia)}$$

Após obter as probabilidades, as notícias agora possuem condições de serem analisadas e classificadas pelo algoritmo.

Além disso, a fim de melhoras o desempenho do classificador, o Suavizador de LaPlace foi utilizado, de modo a corrigir erros causados pelas palavras que não são encontradas na base de dados. E também, removendo as StopWords (palavras que não impactam na notícia, como pronomes e preposições)

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Tratamento de base de treinamento: Limpando o texto e separação da base com base no Target

In [26]:
# Limpando a base Train

train.fillna("",inplace=True)
train['Descrição'] = train['Descrição'].apply(cleanup)
train['Titulo'] = train['Titulo'].apply(cleanup)

# Separando as variáveis de target em categorias

train.Target = train.Target.astype('category')

train.Target.cat.categories = (['Negativa' , 'Neutra', 'Positiva'])

# Separando as noticias pela target

not_positiva = train.loc[train['Target'] == 'Positiva']
not_neutra = train.loc[train['Target'] == 'Neutra']
not_negativa = train.loc[train['Target'] == 'Negativa']


Classificando de acordo com o descrição da notícia:

In [27]:
# Criando uma lista com os descrições de acordo com o target
desc_positiva = list(not_positiva['Descrição'])
desc_neutra = list(not_neutra['Descrição'])
desc_negativa = list(not_negativa['Descrição'])


# Notícias divididas em palavras e sem stopwords
palavras_desc_positiva = stpwd(' '.join(desc_positiva).split())
palavras_desc_neutra = stpwd(' '.join(desc_neutra).split())
palavras_desc_negativa = stpwd(' '.join(desc_negativa).split())


# Criando um conjunto que contém todas as palavras que estão presentes no título
palavras_port_desc = palavras_desc_positiva + palavras_desc_neutra + palavras_desc_negativa
palavras_port_desc = list(dict.fromkeys(palavras_port_desc))
##print(palavras_port_desc)

desc_positiva_series = pd.Series(palavras_desc_positiva)
desc_neutra_series = pd.Series(palavras_desc_neutra)
desc_negativa_series = pd.Series(palavras_desc_negativa)


# Frequencia absoluta que as palavras aparecem nas notícias
desc_positiva_abs = desc_positiva_series.value_counts()
desc_neutras_abs = desc_neutra_series.value_counts()
desc_negativa_abs = desc_negativa_series.value_counts()

##print(desc_positiva_abs)

##print(desc_neutras_abs)

##print(desc_negativa_abs)

In [28]:
# Probabilidades: Positiva, Neutra ou Negativa
prob_positiva = len(palavras_desc_positiva) /len(palavras_port_desc)
prob_neutra = len(palavras_desc_neutra) /len(palavras_port_desc)
prob_negativa = len(palavras_desc_negativa) /len(palavras_port_desc)

# Probabilidade: Positiva, Neutra ou Negativa dada a Notícia
lista_classifica_desc = []
for not_desc in test.Descrição:

    noticia_desc = cleanup(not_desc)
    palavras_desc = stpwd(not_desc.split())

    prob_Positiva_desc = 1
    prob_Neutra_desc = 1
    prob_Negativa_desc = 1

    for p_desc in palavras_desc:
        prob_Positiva_desc = prob_Positiva_desc * lp(p_desc, desc_positiva_abs,palavras_port_desc)

        prob_Neutra_desc = prob_Neutra_desc * lp(p_desc,desc_neutras_abs, palavras_port_desc) 

        prob_Negativa_desc = prob_Negativa_desc * lp(p_desc,desc_negativa_abs,palavras_port_desc) 

    prob_Positiva_desc *= prob_positiva
    prob_Neutra_desc *= prob_neutra
    prob_Negativa_desc *= prob_negativa

# Classificação da notícia com base nas probabilidades de ela ser Positiva, Neutra ou Negativa
    if prob_Positiva_desc > prob_Neutra_desc and prob_Positiva_desc > prob_Negativa_desc:
        lista_classifica_desc.append("Positiva")
    elif prob_Neutra_desc > prob_Positiva_desc and prob_Neutra_desc > prob_Negativa_desc:
        lista_classifica_desc.append("Neutra")
    elif prob_Negativa_desc > prob_Positiva_desc and prob_Negativa_desc > prob_Neutra_desc:
        lista_classifica_desc.append("Negativa")
 
target_desc = pd.Series(lista_classifica_desc)
target_desc.value_counts(normalize=True)
test["Classificação_pela_Descrição"] = target_desc



#test.Classificação_pela_Descrição = test.Classificação_pela_Descrição.astype('category')

#test.Classificação_pela_Descrição.cat.categories = (['Negativa' , 'Neutra', 'Positiva'])


# Comparação das classificações manuais e classificações feitas pelo algorítimo
crosstab_desc_relativo = pd.crosstab(test["Target"], test["Classificação_pela_Descrição"], normalize="index")
crosstab_desc_relativo


Classificação_pela_Descrição,Negativa,Neutra,Positiva
Target,,,
0,0.017241,0.431034,0.551724
1,0.015251,0.823529,0.161220
2,0.013333,0.302222,0.684444


___
### Verificando a performance do Classificador

Com os resultados obtidos podemos verificar como foi o acerto da classificação feita pelo contador.

___
### Porcentagem de *Positivos* (Verdadeiros e Falsos)

In [29]:
# Verdadeiros

positivos_verdadeiros_descrição = crosstab_desc_relativo.iloc[2, 2] * 100
print(positivos_verdadeiros_descrição)

# Falsos

positivos_falsos_descrição = ((crosstab_desc_relativo.iloc[0, 2] + crosstab_desc_relativo.iloc[1, 2])) * 100
print(positivos_falsos_descrição)


68.44444444444444
71.29441815040192


___
### Porcentagem de *Negativos* (Verdadeiros e Falsos)

In [30]:
# Verdadeiros

negativos_verdadeiros_descrição = crosstab_desc_relativo.iloc[0, 0] * 100
print(negativos_verdadeiros_descrição)

# Falsos

negativos_falsos_descrição = ((crosstab_desc_relativo.iloc[1, 0] + crosstab_desc_relativo.iloc[2, 0])) * 100
print(negativos_falsos_descrição)

1.7241379310344827
2.85838779956427


___
### Porcentagem de *Neutros* (Verdadeiros e Falsos)

In [31]:
# Verdadeiros

neutros_verdadeiros_descrição = crosstab_desc_relativo.iloc[1, 1] * 100
print(neutros_verdadeiros_descrição)

# Falsos

neutros_negativos_descrição = ((crosstab_desc_relativo.iloc[0, 1] + crosstab_desc_relativo.iloc[2, 1])) * 100
print(neutros_negativos_descrição)


82.35294117647058
73.32567049808429


___
### *Acurácia*

In [32]:
test.Target = test.Target.astype('category')

test.Target.cat.categories = (['Negativa' , 'Neutra', 'Positiva'])

ac_descrição = (sum(test.Target == test['Classificação_pela_Descrição']) / len(test.Target)) * 100
print(ac_descrição)

ac1_desc = ((neutros_verdadeiros_descrição + negativos_verdadeiros_descrição + positivos_verdadeiros_descrição) / len(test.Target))*100
#(ac1_desc)
    

66.75


___
### Concluindo

___
### Análise dos resultados

O classificador é competente em calcular a probabilidade de uma noticia ser neutra 82.35%, parcialmente preciso ao calcular a probailidade de uma notícia ser positiva 68.44% e impreciso ao tentar achar a mesma coisa para as notícias negativas 1.72%. Posto isso, ao realizarmos uma análise na base dados criada pode-se perceber uma quantidade maior de notícias classificadas como neutras quando comparadas a positivas e negativas, onde houveram mais positivas do que negativas.

Ao analisarmos o cenário econômico global os mercados encontram-se em um processo de recuperação, mas já voltam a acontecer alguns crescimentos em diversos setores. Além disso, muitas notícias traziam informações dentro do aspecto político por estarmos em um ano de eleição e estas foram classificadas como neutras. Estes são alguns fatores que colaboraram para chegar nesse resultado.

___
### Mensagens com dupla negação e sarcasmo

Frases analisadas pelo classificador com dupla negação acabam aumentando a frequência da palavra, coisa que acaba aumentando a tendência da classificação para qual palavra apareceu mais.

---
### Expansão

Para o futuro algo interessante seria aumentar a nossa acurácia e calibrar o contador de uma melhor forma. Isto é, aumentar a base de dados de notícias e tentar diminuir o viés da classificação manual realizada por integrantes distintos.

Ademais, algo interessante de ser implementado seria usar lemmatization, recurso responsável por deixar as palavras encontradas em sua raíz lexical, isso diminui a probabilidade de falhas do contador por considerar "sinônimos" como sendo palavras distintas.

---
### Uso do classificador para aumentar o número de amostras com novas notícias

O contador usará a analise pré-estabelecida com a planilha anterior, assim esse aumento diminuiria a acurácia do mesmo, já que a classificação ficaria dependente da probabilidade existente, reforçando as variáveis com uma maior probabilidade me relação ao outros.

___
### Novos usos

Além do uso para controle de spam em emails mencionado em sala de aula este poderia ser usado para analisar sentimentos de pessoas ou pacientes, coisa que poderia vir a ser útil para o pré diagnóstico de algo sério que precisaria ser encamiado para uma profissional da área de saúde. Outro caso de uso plausível seria gerar um sistema de recomendação para pessoas interessadas em descobrir novos produtos que possuem um comportamento similiar ao que já estão habituadas.

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

[NLP com scikit-learn](https://dadosaocubo.com/nlp-com-scikit-learn/)

[Text pre-processing: Stop words removal using differente libraries](https://towardsdatascience.com/text-pre-processing-stop-words-removal-using-different-libraries-f20bac19929a#:~:text=What%20are%20stop%20words%3F,much%20information%20to%20the%20text)